In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cities = [
    "Warszawa, Polska",
    "Kraków, Polska",
    "Łódź, Polska",
    "Wrocław, Polska",
    "Poznań, Polska",
    "Gdańsk, Polska",
    "Szczecin, Polska",
    "Bydgoszcz, Polska",
    "Lublin, Polska",
    "Białystok, Polska",
    "Katowice, Polska",
    "Gdynia, Polska",
    "Częstochowa, Polska",
    "Radom, Polska",
    "Toruń, Polska",
    "Rzeszów, Polska",
    "Sosnowiec, Polska",
    "Kielce, Polska",
    "Gliwice, Polska",
    "Olsztyn, Polska",
    "Zabrze, Polska",
    "Bielsko-Biała, Polska",
    "Bytom, Polska",
    "Zielona Góra, Polska",
    "Rybnik, Polska",
    "Ruda Śląska, Polska",
    "Opole, Polska",
    "Tychy, Polska",
    "Gorzów Wielkopolski, Polska",
    "Elbląg, Polska",
    "Dąbrowa Górnicza, Polska",
    "Płock, Polska",
    "Wałbrzych, Polska",
    "Włocławek, Polska",
    "Tarnów, Polska",
    "Chorzów, Polska",
    "Koszalin, Polska",
    ##50k
    "Kalisz, Polska",
    "Legnica, Polska",
    "Grudziądz, Polska",
    "Jaworzno, Polska",
    "Słupsk, Polska",
    "Jastrzębie-Zdrój, Polska",
    "Nowy Sącz, Polska",
    "Jelenia Góra, Polska",
    "Siedlce, Polska",
    "Mysłowice, Polska",
    "Konin, Polska",
    "Piła, Polska",
    "Piotrków Trybunalski, Polska",
    "Lubin, Polska",
    "Inowrocław, Polska",
    "Ostrów Wielkopolski, Polska",
    "Suwałki, Polska",
    "Stargard, Polska",
    "Gniezno, Polska",
    "Ostrowiec Świętokrzyski, Polska",
    "Siemianowice Śląskie, Polska",
    "Głogów, Polska",
    "Pabianice, Polska",
    "Leszno, Polska",
    "Żory, Polska",
    "Zamość, Polska",
    "Pruszków, Polska",
    "Łomża, Polska",
    "Ełk, Polska",
    "Tarnowskie Góry, Polska",
    "Tomaszów Mazowiecki, Polska",
    "Chełm, Polska",
    "Mielec, Polska",
    "Kędzierzyn-Koźle, Polska",
    "Przemyśl, Polska",
    "Stalowa Wola, Polska",
    "Tczew, Polska",
    "Biała Podlaska, Polska",
    "Bełchatów, Polska",
    "Świdnica, Polska",
    "Będzin, Polska",
    "Zgierz, Polska",
    "Piekary Śląskie, Polska",
    "Racibórz, Polska",
    "Legionowo, Polska",
    "Ostrołęka, Polska",
]

In [3]:
num_people = [
    1794166,
    779966,
    672185,
    641928,
    532048,
    470805,
    398255,
    344091,
    338586,
    296958,
    290553,
    244969,
    217530,
    209296,
    198613,
    197863,
    197586,
    193415,
    177049,
    171249,
    170924,
    169756,
    163255,
    140892,
    137128,
    136423,
    127839,
    126871,
    122589,
    118582,
    118285,
    118268,
    109971,
    108561,
    107498,
    106846,
    106235,
    99106,
    98436,
    93564,
    90368,
    89780,
    88038,
    83558,
    78335,
    77813,
    74559,
    72539,
    72527,
    72250,
    71710,
    71674,
    71560,
    69639,
    67579,
    67570,
    67404,
    66270,
    66120,
    63945,
    62854,
    62844,
    62785,
    62623,
    62573,
    61903,
    61756,
    61338,
    61135,
    60075,
    60021,
    59779,
    59623,
    59430,
    56942,
    56419,
    56222,
    56008,
    55673,
    54702,
    54259,
    53529,
    51656,
]

In [11]:
RESOLUTION = 9

In [4]:
import pandas as pd
df = pd.DataFrame({"city": cities, "num_people": num_people})
def get_stratify_index(num_people) -> int:
    if num_people >= 500_000:
        return 0
    elif num_people >= 200_000:
        return 1
    elif num_people >= 100_000:
        return 2
    else:
        return 3
df["stratify"] = df["num_people"].apply(get_stratify_index)
df

,city,num_people,stratify
0,"Warszawa, Polska",1794166,0
1,"Kraków, Polska",779966,0
2,"Łódź, Polska",672185,0
3,"Wrocław, Polska",641928,0
4,"Poznań, Polska",532048,0
...,...,...,...
78,"Zgierz, Polska",55673,3
79,"Piekary Śląskie, Polska",54702,3
80,"Racibórz, Polska",54259,3
81,"Legionowo, Polska",53529,3


In [5]:
from srai.regionalizers import H3Regionalizer
from tqdm.auto import tqdm
from srai.utils import geocode_to_region_gdf

In [6]:
areas = [
    geocode_to_region_gdf(city) for city in tqdm(cities)
]

  0%|          | 0/83 [00:00<?, ?it/s]

In [11]:
# import pandas as pd

# pd.concat(areas).explore()

In [7]:
import geopandas as gpd
from typing import List

def get_regions(cities: List[str], resolution: int) -> gpd.GeoDataFrame:
    areas = [
        geocode_to_region_gdf(city) for city in tqdm(cities)
    ]
    regionizer = H3Regionalizer(resolution=resolution)
    regions_gdf = regionizer.transform(pd.concat(areas))
    return regions_gdf

In [8]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["stratify"])
train_df = train_df.sort_values(by="num_people", ascending=False)
val_df = val_df.sort_values(by="num_people", ascending=False)

In [9]:
train_cities = train_df["city"].tolist()
val_cities = val_df["city"].tolist()

In [12]:
train_regions_gdf = get_regions(train_cities, resolution=RESOLUTION)
val_regions_gdf = get_regions(val_cities, resolution=RESOLUTION)

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [14]:
from srai.neighbourhoods import H3Neighbourhood

train_neighbourhood = H3Neighbourhood(train_regions_gdf)
val_neighbourhood = H3Neighbourhood(val_regions_gdf)

In [15]:
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER

In [16]:
from srai.embedders.hex2vec import Hex2VecEmbedder
from functional import seq


expected_output_features = seq(HEX2VEC_FILTER.items()).starmap(lambda k, v: list(map(lambda x: f"{k}_{x}", v))).flatten().list()
expected_output_features

['aeroway_aerodrome',
 'aeroway_apron',
 'aeroway_gate',
 'aeroway_hangar',
 'aeroway_helipad',
 'aeroway_heliport',
 'aeroway_navigationaid',
 'aeroway_runway',
 'aeroway_taxiway',
 'aeroway_terminal',
 'aeroway_windsock',
 'amenity_animal_boarding',
 'amenity_animal_breeding',
 'amenity_animal_shelter',
 'amenity_arts_centre',
 'amenity_atm',
 'amenity_baby_hatch',
 'amenity_bank',
 'amenity_bar',
 'amenity_bbq',
 'amenity_bench',
 'amenity_bicycle_parking',
 'amenity_bicycle_rental',
 'amenity_bicycle_repair_station',
 'amenity_biergarten',
 'amenity_boat_rental',
 'amenity_boat_sharing',
 'amenity_brothel',
 'amenity_bureau_de_change',
 'amenity_bus_station',
 'amenity_cafe',
 'amenity_car_rental',
 'amenity_car_sharing',
 'amenity_car_wash',
 'amenity_casino',
 'amenity_charging_station',
 'amenity_childcare',
 'amenity_cinema',
 'amenity_clinic',
 'amenity_clock',
 'amenity_college',
 'amenity_community_centre',
 'amenity_conference_centre',
 'amenity_courthouse',
 'amenity_crema

In [18]:
def get_features(cities: List[str]) -> gpd.GeoDataFrame:
    features = [
        gpd.read_parquet(f"data/raw/{city}.parquet") for city in tqdm(cities)
    ]
    features = pd.concat(features)
    features = features[~features.index.duplicated(keep='first')]
    return features

In [19]:
train_features = get_features(train_cities)
val_features = get_features(val_cities)

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [20]:
train_regions_gdf

,geometry
region_id,
891f53d9883ffff,"POLYGON ((20.94988 52.35291, 20.94734 52.35374..."
891f53ca6cfffff,"POLYGON ((20.99257 52.32545, 20.99517 52.32626..."
891f53514c3ffff,"POLYGON ((21.07033 52.13857, 21.07026 52.13691..."
891f53525cfffff,"POLYGON ((21.16832 52.22027, 21.17092 52.22109..."
891f5226bcbffff,"POLYGON ((20.99968 52.16874, 21.00227 52.16956..."
...,...
891f50308c7ffff,"POLYGON ((21.54940 53.10493, 21.54933 53.10329..."
891f5030aabffff,"POLYGON ((21.54602 53.08778, 21.54594 53.08614..."
891f503549bffff,"POLYGON ((21.62243 53.10924, 21.62235 53.10761..."


In [21]:
from srai.joiners import IntersectionJoiner

joiner = IntersectionJoiner()
train_joint = joiner.transform(train_regions_gdf, train_features)
val_joint = joiner.transform(val_regions_gdf, val_features)

In [22]:
experiment_model_sizes = [
    [256, 128, 64],
    [128, 64],
    [1024, 512],
    [2048, 1024],
    [2048, 1024, 512],
    [256, 128, 64, 32],
]

In [26]:
EPOCHS = 20

In [27]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
from srai.embedders import Hex2VecEmbedder

for sizes in experiment_model_sizes:
    checkpoint_callback = ModelCheckpoint(save_top_k=2, monitor="val_f1", mode="max", filename="{epoch}-{val_f1:.2f}", dirpath=f"models_{RESOLUTION}/{sizes}")
    embedder = Hex2VecEmbedder(encoder_sizes=sizes, expected_output_features=expected_output_features)
    logger = WandbLogger(project=f"hex2vec_pl_r{RESOLUTION}_b1024_50k", name=f"model_sizes={sizes}")
    embedder.fit_transform(train_regions_gdf, train_features, train_joint, train_neighbourhood, val_regions_gdf, val_features, val_joint, val_neighbourhood, batch_size=1024, trainer_kwargs={"max_epochs": EPOCHS, "accelerator": "gpu", "logger": logger, "callbacks": [checkpoint_callback]})
    wandb.finish()

100%|██████████| 81999/81999 [00:02<00:00, 35528.44it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
100%|██████████| 17613/17613 [00:00<00:00, 33277.15it/s]
You are using a CUDA device ('NVIDIA GeForce RTX 4080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/swozniak/projects/srai/.venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/swozniak/projects/srai/models_9/[256, 128, 64] exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    

Sanity Checking: 0it [00:00, ?it/s]

/home/swozniak/projects/srai/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/swozniak/projects/srai/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_f1_epoch,▁▅▆▆▆▇▇▇▇▇▇▇████████
train_f1_step,▁▄▄▄▄▅▅▆▆▆▆▆▆▅▆▇▆▇▇▆▆▇▆▇▇▇█▆▆▇▆▇▆▆▆█▇▆█▆
train_loss_epoch,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_loss_step,█▆▅▅▅▄▃▃▂▃▃▃▂▃▃▂▃▂▂▂▃▃▃▂▂▂▁▃▂▂▃▂▃▂▂▂▂▃▁▃
trainer/global_step,▁▁▁▁▂▁▁▁▃▁▁▁▁▂▂▂▂▂▄▂▂▂▅▂▂▂▆▂▂▂▂▂▂▂▂▂█▃▃▃
val_f1_epoch,▁▃▅▅▅▆▇▇▇▇█▇▇███▇███
val_f1_step,▁▃▃▂▇▄▄▃▄▅▅▇▅▄▇▅▆▄▆▅▅▇▆▆▃▅▇▇▆▇▇█▅▃▇▅▆▃▆▆
val_loss_epoch,█▅▅▄▄▅▂▄▄▃▁▂▂▂▂▁▃▂▂▁
val_loss_step,█▇█▅▄▆▅▆▅▄▇▅▅▅▄▅▅▆▆▆▆▂▄▃▄▃▅▃▅▃▃▁▄▆▄▅▄▆▄▄
epoch,19


In [33]:
import torch
with open("models/[256, 128, 64]/epoch=4-val_f1=0.84.ckpt", "rb") as f:
    ble = torch.load(f)
ble

{'epoch': 4,
 'global_step': 15660,
 'pytorch-lightning_version': '2.0.2',
 'state_dict': OrderedDict([('encoder.0.weight',
               tensor([[ 0.1185, -0.0679, -0.0759,  ...,  0.1910, -0.0199,  0.0439],
                       [ 0.0261, -0.1147, -0.0818,  ..., -0.0096,  0.1501,  0.0937],
                       [ 0.2804, -0.0689, -0.0469,  ...,  0.1040,  0.0130, -0.1599],
                       ...,
                       [-0.1363, -0.0415, -0.0739,  ..., -0.1396, -0.0171,  0.1324],
                       [-0.1597, -0.0621, -0.0465,  ...,  0.0469,  0.0422, -0.1400],
                       [-0.1683,  0.0277, -0.1053,  ..., -0.1481,  0.2068,  0.0496]],
                      device='cuda:0')),
              ('encoder.0.bias',
               tensor([-1.3661e-02, -8.9733e-03, -1.4578e-04,  2.4494e-02,  9.3286e-03,
                       -5.2716e-02,  1.5052e-01, -2.5846e-01,  1.4060e-01, -1.0114e-01,
                        1.7142e-01,  1.0915e-01, -9.3597e-02, -1.4510e-02,  1.2225e-01,

In [34]:
embedder._model.load_state_dict(ble["state_dict"])

<All keys matched successfully>

In [35]:
embedder.save("embedder_50k_no_bikes")

In [20]:
raise ValueError

ValueError: 

In [ ]:
wro_area = geocode_to_region_gdf("Wrocław, Polska")
wro_regions = H3Regionizer(resolution=10).transform(wro_area)

In [ ]:
wro_features = gpd.read_parquet("data/raw/Wrocław, Polska.parquet")
wro_joint = joiner.transform(wro_regions, wro_features)

In [ ]:
embeddings = embedder.transform(wro_regions, wro_features, wro_joint)
embeddings

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
region_id,,,,,,,,,,,,,,,,,,,,,
8a1e2042019ffff,0.173556,0.288988,0.048884,0.295544,-0.381908,0.176133,-0.449802,0.089579,0.383911,0.196264,...,-0.154303,0.115035,0.443080,0.278701,-0.177178,0.440510,0.042160,-0.166764,0.857604,0.265572
8a1e2040e4c7fff,0.569120,-0.269827,-0.026317,0.138060,0.303489,0.177815,-0.224048,0.628205,0.362004,-0.376314,...,0.446630,0.482390,0.026622,-0.041049,-0.581953,-0.223292,0.333704,0.174272,0.054007,0.327932
8a1e204e4057fff,0.116785,-0.110424,0.035945,0.420315,0.147762,0.444220,-0.189481,-0.118806,0.144089,-0.606665,...,0.680880,0.405082,0.180536,-0.008628,0.020352,-0.109906,-0.094148,-0.110837,-0.249448,0.363240
8a1e2040a857fff,-0.174507,0.335021,-0.044738,0.184399,-0.130447,0.036740,0.000599,0.494398,-0.314822,-0.505360,...,0.118521,-0.206467,-0.393964,0.636533,-0.712550,0.087959,0.680310,0.801508,-0.370114,-0.547396
8a1e20409987fff,0.024551,0.387958,-0.330577,0.063520,-0.216052,0.465252,0.269703,-0.127062,-0.678566,-0.356191,...,0.152853,-0.394317,0.273484,0.336321,-0.575704,0.525373,0.018739,0.823310,-0.849898,-0.920894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1e2040b15ffff,0.044890,-0.369053,0.119655,0.062009,0.310631,0.285013,-0.399880,0.677543,-0.033250,-0.578567,...,-0.000411,0.254198,-0.384687,0.339833,-0.660461,-0.392639,0.371422,0.184200,-0.597203,-0.451025
8a1e20425017fff,-0.321903,0.463967,0.170445,-0.350112,0.230916,-0.205377,0.548455,0.685074,0.510162,0.676436,...,0.546984,0.338335,-0.094369,-0.529648,0.397240,0.241065,0.272673,0.214156,-0.562904,-0.122838
8a1e20431257fff,-0.733405,0.116287,-0.143508,0.058409,0.062472,0.137480,0.200453,0.830312,-0.078465,0.415744,...,-0.213926,-0.603942,0.375143,-0.624577,0.141776,-0.395584,-0.335985,0.149248,0.190093,-0.162821


In [ ]:
from sklearn.cluster import KMeans
from srai.plotting import plot_numeric_data

In [ ]:
cluster = KMeans(n_clusters=10)
clusters = cluster.fit_predict(embeddings)
clusters

/home/swozniak/projects/srai/.venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([1, 1, 1, ..., 7, 3, 9], dtype=int32)

In [ ]:
plot_gdf = wro_regions.copy()
plot_gdf["cluster"] = clusters

plot_numeric_data(wro_regions, plot_gdf, "cluster", colormap="tab10")

In [ ]:
from srai.embedders import CountEmbedder

count_embedder = CountEmbedder(count_subcategories=False)

count_embeddings = count_embedder.transform(wro_regions, wro_features, wro_joint)
count_embeddings

,aeroway,amenity,building,healthcare,historic,landuse,leisure,military,natural,office,shop,sport,tourism,water,waterway
region_id,,,,,,,,,,,,,,,
8a1e20475787fff,0,1,0,0,0,2,1,0,0,0,0,0,0,0,0
8a1e2042e437fff,0,0,1,0,0,4,1,0,1,0,0,0,0,0,0
8a1e20471a77fff,0,2,0,0,0,2,0,0,0,0,0,0,1,0,1
8a1e20450567fff,0,0,0,0,0,0,0,0,2,0,0,0,0,1,1
8a1e204e611ffff,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1e20472a8ffff,0,1,2,0,0,2,1,0,0,0,0,0,0,0,0
8a1e20556507fff,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8a1e204e5177fff,0,0,26,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.cluster import KMeans
from srai.plotting import plot_numeric_data

In [ ]:
cluster = KMeans(n_clusters=10)
clusters = cluster.fit_predict(count_embeddings)
plot_gdf = wro_regions.copy()
plot_gdf["cluster"] = clusters
plot_numeric_data(wro_regions, plot_gdf, "cluster", colormap="tab10")

In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
embedder = Hex2VecEmbedder.load(Path("r10_pl_100k"))

In [ ]:
from srai.joiners import IntersectionJoiner
from srai.regionizers import H3Regionizer
joiner = IntersectionJoiner()

In [ ]:
area = geocode_to_region_gdf("Gdańsk, Polska")
regions = H3Regionizer(resolution=10).transform(area)
features = gpd.read_parquet("data/raw/Gdańsk, Polska.parquet")
joint = joiner.transform(regions, features)

In [ ]:
embeddings = embedder.transform(regions, features, joint)
embeddings

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
region_id,,,,,,,,,,,,,,,,,,,,,
8a1f724e125ffff,0.231239,-0.196225,0.224774,-0.204577,0.272633,0.339952,0.119712,-0.009222,0.140706,-0.017214,...,-0.186627,0.006933,-0.003167,0.170422,0.066978,0.155175,-0.002860,0.052813,0.157953,-0.415896
8a1f724e5aa7fff,0.231239,-0.196225,0.224774,-0.204577,0.272633,0.339952,0.119712,-0.009222,0.140706,-0.017214,...,-0.186627,0.006933,-0.003167,0.170422,0.066978,0.155175,-0.002860,0.052813,0.157953,-0.415896
8a1f72498d2ffff,-0.357695,-0.133341,0.298338,-0.660436,0.296341,0.297462,-0.129616,-0.105311,0.297201,0.434056,...,-0.091607,-0.357733,0.301568,-0.043104,-0.410060,-0.551953,0.133317,0.425072,-0.048675,0.239983
8a1f7248c177fff,-0.286190,0.011620,0.180642,-0.288589,-0.002698,-0.037762,-0.320312,0.084955,0.086787,-0.191180,...,0.047522,0.052893,0.430177,0.054441,-0.542808,-0.310894,-0.151308,0.275078,0.050520,-0.078827
8a1f724c0937fff,0.231239,-0.196225,0.224774,-0.204577,0.272633,0.339952,0.119712,-0.009222,0.140706,-0.017214,...,-0.186627,0.006933,-0.003167,0.170422,0.066978,0.155175,-0.002860,0.052813,0.157953,-0.415896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1f0986c957fff,0.181489,0.382292,0.154497,0.007077,-0.205120,0.086614,0.175767,0.023460,0.363786,-0.154332,...,0.363855,-0.192280,-0.096331,-0.182033,0.137739,0.162328,-0.273333,-0.062072,-0.035979,-0.111762
8a1f7248cd77fff,0.071181,0.587175,-0.045538,-0.300783,-0.064654,0.072767,-0.338590,-0.750539,-0.104153,0.203407,...,-0.498614,0.224562,0.368970,0.182191,-0.056294,-0.289853,0.172294,0.298320,-0.160956,-0.179076
8a1f09b44da7fff,-0.408609,0.143905,-0.246527,-0.177357,0.290274,-0.370556,0.320862,0.267324,0.071227,0.264836,...,-0.598449,0.115370,-0.324746,-0.012693,0.133477,-0.468858,0.262236,-0.069581,0.043581,-0.447354


In [ ]:
from sklearn.cluster import KMeans
from srai.plotting import plot_numeric_data
cluster = KMeans(n_clusters=10)
clusters = cluster.fit_predict(embeddings)
clusters
plot_gdf = regions.copy()
plot_gdf["cluster"] = clusters

plot_numeric_data(regions, plot_gdf, "cluster", colormap="tab10")

In [ ]:
from srai.embedders import CountEmbedder

count_embedder = CountEmbedder(expected_output_features=expected_output_features)

count_embeddings = count_embedder.transform(wro_regions, wro_features, wro_joint)
count_embeddings

,aeroway_aerodrome,aeroway_apron,aeroway_gate,aeroway_hangar,aeroway_helipad,aeroway_heliport,aeroway_navigationaid,aeroway_runway,aeroway_taxiway,aeroway_terminal,...,waterway_lock_gate,waterway_pressurised,waterway_river,waterway_riverbank,waterway_stream,waterway_tidal_channel,waterway_turning_point,waterway_water_point,waterway_waterfall,waterway_weir
region_id,,,,,,,,,,,,,,,,,,,,,
8a1e20475787fff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8a1e2042e437fff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8a1e20471a77fff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8a1e20450567fff,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8a1e204e611ffff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1e20472a8ffff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8a1e20556507fff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8a1e204e5177fff,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from umap import UMAP

umapped = UMAP(n_components=64).fit_transform(count_embeddings)

In [ ]:
cluster = KMeans(n_clusters=10)
clusters = cluster.fit_predict(umapped)
clusters
plot_gdf = wro_regions.copy()
plot_gdf["cluster"] = clusters

plot_numeric_data(wro_regions, plot_gdf, "cluster", colormap="tab10")

In [ ]:
val_cities

['Kielce, Polska',
 'Radom, Polska',
 'Poznań, Polska',
 'Elbląg, Polska',
 'Chorzów, Polska',
 'Ruda Śląska, Polska',
 'Szczecin, Polska',
 'Opole, Polska']

In [ ]:
area = geocode_to_region_gdf("Poznań, Polska")
regions = H3Regionizer(resolution=10).transform(area)
features = gpd.read_parquet("data/raw/Poznań, Polska.parquet")
joint = joiner.transform(regions, features)

In [ ]:
embeddings = embedder.transform(regions, features, joint)
embeddings

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
region_id,,,,,,,,,,,,,,,,,,,,,
8a1e24a1104ffff,-0.230435,0.146009,0.030841,-0.187705,-0.209398,0.106841,-0.325555,0.157971,-0.209766,-0.359187,...,0.260583,-0.252027,-0.202288,0.206585,-0.247984,-0.036301,-0.109473,-0.269359,0.504744,0.314436
8a1e24aaa9affff,-0.231022,0.096924,-0.389670,-0.391343,0.146649,-0.003101,0.007737,0.083627,0.065311,-0.191573,...,-0.173618,-0.001557,-0.073462,0.141525,0.201009,-0.348623,0.320697,-0.457041,-0.224055,-0.434583
8a1e24b8ba9ffff,-0.590725,-0.655228,0.334008,-0.066888,0.020288,0.448003,-0.534694,-0.039649,-0.104514,0.210112,...,0.076609,0.080391,-0.370678,0.092443,0.070986,-0.616261,-0.207067,0.145081,0.484168,0.529770
8a1e24b819affff,0.451259,0.103240,-0.147869,0.249194,0.125329,-0.208991,0.003606,-0.137312,-0.037852,0.064119,...,0.286621,-0.158282,-0.030330,-0.047827,0.134048,0.076079,-0.085485,0.167294,0.107046,0.529015
8a1e24a85337fff,-0.242026,-0.205650,-0.079857,-0.007808,0.327113,0.260520,-0.153551,0.245878,-0.675985,-0.277530,...,-0.261230,-0.222018,0.336282,-0.465637,0.021872,0.189414,-0.096297,0.212082,-0.319121,-0.182670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1e24a0260ffff,-0.325921,0.068947,0.257563,0.255301,-0.121066,-0.080549,-0.019385,0.577814,-0.559995,-0.132585,...,0.230132,0.172570,0.185074,-0.159591,-0.350715,-0.280529,-0.017984,-0.271788,0.141725,0.270948
8a1e24af674ffff,0.020054,0.210567,0.099235,0.244769,0.177670,-0.555532,0.463252,-0.347973,0.185183,-0.202271,...,0.242616,-0.276998,0.163584,0.217420,0.360064,-0.417928,0.200024,0.401340,0.333782,0.305902
8a1e24a125affff,-0.379483,-0.503757,-0.279412,-0.175121,-0.036091,0.288348,-0.254106,0.585136,0.489879,0.500766,...,-0.539271,0.430331,0.380616,0.887893,0.860620,1.065669,-0.275282,0.419437,-0.003069,-0.319620


In [ ]:
cluster = KMeans(n_clusters=10)
clusters = cluster.fit_predict(embeddings)
plot_gdf = regions.copy()
plot_gdf["cluster"] = clusters

plot_numeric_data(regions, plot_gdf, "cluster", colormap="tab10")